In [8]:
import os
from pathlib import Path

import IPython.display as ipd
import librosa
import matplotlib
from matplotlib import pyplot as plt
from numba import jit
import numpy as np
import pandas as pd
import scipy

import sys
sys.path.append('..')

%matplotlib inline

In [14]:
# Load the data
fn_wav = Path('..', 'data', 'datasets', 'raw/audio', 'YTB-005.wav')
Fs = 22050
x, Fs = librosa.load(fn_wav, sr=Fs)

# Compute Magnitude STFT
N = 4096
H = 1024
# STFT of fn_wav - returns complex-valued matrix
X = librosa.stft(x, n_fft=N, hop_length=H)



print(X.shape)
print(X.dtype)

print(X[1000:1004, 1000:1004])


(2049, 3779)
complex64
[[-0.02205317-9.86633524e-02j -0.02472674-8.31711292e-02j
  -0.10832526-1.48239478e-01j -0.18287826-2.08799899e-01j]
 [-0.03268886+4.98580411e-02j  0.02525608+4.21542265e-02j
   0.09590914-3.86860929e-02j  0.13075086+1.79273114e-01j]
 [ 0.10387797+5.12186065e-02j -0.03470065-5.80120832e-02j
   0.04832032+1.39560446e-01j -0.05475457-1.77558661e-01j]
 [-0.142356  -1.24395736e-01j -0.07921167+1.93568005e-04j
  -0.1683535 -4.78123836e-02j -0.05312799+1.62806451e-01j]]


In [ ]:
# Various ways to print, view, and display X

# 1. Basic info and statistics
print("=== Basic Info ===")
print(f"Shape: {X.shape}")
print(f"Dtype: {X.dtype}")
print(f"Size: {X.size:,} elements")
print(f"Memory: {X.nbytes / 1024**2:.2f} MB")

# 2. Statistics (on magnitude)
X_mag = np.abs(X)
print("\n=== Magnitude Statistics ===")
print(f"Min magnitude: {X_mag.min():.6f}")
print(f"Max magnitude: {X_mag.max():.6f}")
print(f"Mean magnitude: {X_mag.mean():.6f}")
print(f"Std magnitude: {X_mag.std():.6f}")

# 3. Print with different numpy options
print("\n=== Different Print Styles ===")
print("First 3x3 slice (default):")
print(X[:3, :3])
print("\nFirst 3x3 slice (suppress scientific notation):")
np.set_printoptions(suppress=True, precision=4)
print(X[:3, :3])
np.set_printoptions()  # reset

# 4. View magnitude and phase separately
print("\n=== Magnitude and Phase ===")
print("Magnitude (first 3x3):")
print(np.abs(X[:3, :3]))
print("\nPhase in radians (first 3x3):")
print(np.angle(X[:3, :3]))
print("\nPhase in degrees (first 3x3):")
print(np.angle(X[:3, :3]) * 180 / np.pi)

# 5. Power spectrogram (magnitude squared)
X_power = np.abs(X)**2
print("\n=== Power Spectrogram ===")
print(f"Power range: {X_power.min():.6e} to {X_power.max():.6e}")
print("First 3x3 power values:")
print(X_power[:3, :3])

# 6. Decibel scale (log magnitude)
X_db = librosa.amplitude_to_db(X_mag)
print("\n=== Decibel Scale ===")
print(f"dB range: {X_db.min():.2f} to {X_db.max():.2f} dB")
print("First 3x3 dB values:")
print(X_db[:3, :3])

# 7. Summary by frequency/time
print("\n=== Summary Statistics ===")
print("Mean magnitude per frequency bin (first 10 bins):")
print(X_mag[:10, :].mean(axis=1))
print("\nMean magnitude per time frame (first 10 frames):")
print(X_mag[:, :10].mean(axis=0))

In [ ]:
# Visualizations of X

# 1. Magnitude spectrogram (linear scale)
X_mag = np.abs(X)
plt.figure(figsize=(12, 6))
plt.imshow(X_mag, aspect='auto', origin='lower', cmap='viridis')
plt.colorbar(label='Magnitude')
plt.xlabel('Time Frame')
plt.ylabel('Frequency Bin')
plt.title('Magnitude Spectrogram (Linear Scale)')
plt.tight_layout()
plt.show()

# 2. Power spectrogram (magnitude squared)
X_power = np.abs(X)**2
plt.figure(figsize=(12, 6))
plt.imshow(X_power, aspect='auto', origin='lower', cmap='hot')
plt.colorbar(label='Power')
plt.xlabel('Time Frame')
plt.ylabel('Frequency Bin')
plt.title('Power Spectrogram')
plt.tight_layout()
plt.show()

# 3. Decibel spectrogram (log scale) - most common
X_db = librosa.amplitude_to_db(X_mag)
plt.figure(figsize=(12, 6))
plt.imshow(X_db, aspect='auto', origin='lower', cmap='magma')
plt.colorbar(label='Magnitude (dB)')
plt.xlabel('Time Frame')
plt.ylabel('Frequency Bin')
plt.title('Magnitude Spectrogram (dB Scale)')
plt.tight_layout()
plt.show()

# 4. Using librosa's built-in display (with proper time/freq axes)
T_coef = librosa.frames_to_time(np.arange(X.shape[1]), sr=Fs, hop_length=H)
F_coef = librosa.fft_frequencies(sr=Fs, n_fft=N)

plt.figure(figsize=(12, 6))
librosa.display.specshow(X_db, x_axis='time', y_axis='hz', 
                         sr=Fs, hop_length=H, cmap='magma')
plt.colorbar(label='Magnitude (dB)')
plt.title('Spectrogram with Time/Frequency Axes')
plt.tight_layout()
plt.show()

# 5. Phase spectrogram
X_phase = np.angle(X)
plt.figure(figsize=(12, 6))
plt.imshow(X_phase, aspect='auto', origin='lower', cmap='hsv')
plt.colorbar(label='Phase (radians)')
plt.xlabel('Time Frame')
plt.ylabel('Frequency Bin')
plt.title('Phase Spectrogram')
plt.tight_layout()
plt.show()

# 6. Real and Imaginary parts separately
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
im0 = axes[0].imshow(X.real, aspect='auto', origin='lower', cmap='RdBu')
axes[0].set_title('Real Part')
axes[0].set_xlabel('Time Frame')
axes[0].set_ylabel('Frequency Bin')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(X.imag, aspect='auto', origin='lower', cmap='RdBu')
axes[1].set_title('Imaginary Part')
axes[1].set_xlabel('Time Frame')
axes[1].set_ylabel('Frequency Bin')
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Additional viewing options

# 1. View as pandas DataFrame (useful for small slices)
print("=== Pandas DataFrame View ===")
# View a small slice as DataFrame
X_mag_slice = np.abs(X[:10, :10])
df = pd.DataFrame(X_mag_slice, 
                  index=[f'{f:.1f} Hz' for f in F_coef[:10]],
                  columns=[f'{t:.2f}s' for t in T_coef[:10]])
print(df)

# 2. View specific frequency/time ranges
print("\n=== Specific Ranges ===")
# View frequency range around 1000 Hz
freq_idx = np.argmin(np.abs(F_coef - 1000))
print(f"Frequency bin {freq_idx} = {F_coef[freq_idx]:.1f} Hz")
print(f"Magnitude at this frequency (first 10 frames):")
print(np.abs(X[freq_idx, :10]))

# View time range around 5 seconds
time_idx = np.argmin(np.abs(T_coef - 5.0))
print(f"\nTime frame {time_idx} = {T_coef[time_idx]:.2f} seconds")
print(f"Magnitude at this time (first 10 frequency bins):")
print(np.abs(X[:10, time_idx]))

# 3. Histogram of magnitude values
plt.figure(figsize=(10, 6))
plt.hist(X_mag.flatten(), bins=100, log=True, edgecolor='black', alpha=0.7)
plt.xlabel('Magnitude')
plt.ylabel('Count (log scale)')
plt.title('Distribution of STFT Magnitude Values')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 4. Slicing and indexing examples
print("\n=== Slicing Examples ===")
print("Every 100th frequency bin, every 50th time frame:")
print(f"Shape: {X[::100, ::50].shape}")
print("Last 100 frequency bins, last 100 time frames:")
print(f"Shape: {X[-100:, -100:].shape}")

# 5. Check for NaN or Inf values
print("\n=== Data Quality Check ===")
print(f"NaN values: {np.isnan(X).sum()}")
print(f"Inf values: {np.isinf(X).sum()}")
print(f"Zero values: {(X == 0).sum()}")

In [ ]:
# Get time and frequency coefficients
# Time axis: convert frame indices to time in seconds
T_coef = librosa.frames_to_time(np.arange(X.shape[1]), sr=Fs, hop_length=H)
# Frequency axis: get frequency bins in Hz
F_coef = librosa.fft_frequencies(sr=Fs, n_fft=N)
